In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K

In [3]:
Height = 320
Width = 320


In [5]:
import sys
sys.path.insert(1, '../input/customunett/custom_unet.py')
from custom_unet import build_unet
customunet17_model = build_unet((Height, Width, 3))

In [6]:
customunet17_model.load_weights('../input/model-customunet/2017custom_unet.h5')


In [7]:
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

def shuffle_data(image, mask):
    image, mask = shuffle(image, mask, random_state=42)
    return image, mask
def data_loading(data_path, split=0.25):
    images = sorted(glob(os.path.join(data_path, "trainx", "*.jpg")))
    masks = sorted(glob(os.path.join(data_path, "trainy", "*.png")))

    split_ratio = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_ratio, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_ratio, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_ratio, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_ratio, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def load_image(img_path):
    img_path = img_path.decode()
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height))
    img = img/255.0
    img = img.astype(np.float32)
    return img

def load_mask(mask_path):
    mask_path = mask_path.decode()
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (Width, Height))
    mask = mask/255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

def tf_parsing(image, mask):
    def _parse(image, mask):
        image = load_image(image)
        mask = load_mask(mask)
        return image, mask

    image, mask = tf.numpy_function(_parse, [image, mask], [tf.float32, tf.float32])
    image.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return image, mask

def TF_dataset(images, masks, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.map(tf_parsing)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(10)
    return dataset

In [8]:
dataset_path = "../input"
train_path = os.path.join(dataset_path, "isic2017-and-ph2/ISIC_2017 + PH2/ISIC_2017")

(Xtrain, Ytrain), (Xvalid, Yvalid), (Xtest, Ytest) = data_loading(train_path)

print(f"Test: {len(Xtest)} - {len(Ytest)}")

test_dataset = TF_dataset(Xtest, Ytest, batch_size=1)

Test: 500 - 500


In [9]:
customunet_pred = customunet17_model.predict(test_dataset)

In [10]:
test_y = np.concatenate([y for x, y in test_dataset], axis=0)

In [11]:
from tensorflow.keras.metrics import Recall, Precision
def iou(y_true, y_pred): 
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    iou_value = (intersection + 1e-15) / (union + 1e-15)
    iou_value= iou_value.astype(np.float32)
    return iou_value
smooth = 1e-15
def dice_coeff(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


In [12]:
print("TEST_IOU for Cusmtom UNET ISIC 2017",iou(test_y, customunet_pred))

TEST_IOU for Cusmtom UNET ISIC 2017 0.8246765


In [13]:
print("TEST_DICE for Cusmtom UNET ISIC 2017",dice_coeff(test_y, customunet_pred).numpy())

TEST_DICE for Cusmtom UNET ISIC 2017 0.903915


In [15]:
cutoff = 0.5                              
y_pred_classes = np.zeros_like(customunet_pred)
y_pred_classes[customunet_pred > cutoff] = 1
                            
y_true_classes = np.zeros_like(test_y)
y_true_classes[test_y > cutoff] = 1

In [16]:
m = tf.keras.metrics.Recall()
m.update_state(test_y.flatten(),customunet_pred.flatten())
print('TEST_Recall for Cusmtom UNET ISIC 2017', m.result().numpy())

TEST_Recall for Cusmtom UNET ISIC 2017 0.8991161


In [17]:
m = tf.keras.metrics.Precision()
m.update_state(test_y.flatten(),customunet_pred.flatten())
print('TEST_Precision for Cusmtom UNET ISIC 2017', m.result().numpy())

TEST_Precision for Cusmtom UNET ISIC 2017 0.91212803


In [18]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (Height, Width))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (Height, Width))
    ori_x = x
    x = x/np.max(x)
    x = x.astype(np.int32)
    return ori_x, x

def save_result(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((Height, 10, 3)) * 255.0

    ori_y = np.expand_dims(ori_y, axis=-1) 
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) 
    y_pred = np.expand_dims(y_pred, axis=-1)   
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0 

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [19]:
make_dir("results")
SCORE = []
for x, y in tqdm(zip(Xtest, Ytest), total=len(Xtest)):
    name = x.split("/")[-1]

     
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

        
    y_pred = customunet17_model.predict(x)[0] > 0.5
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred.astype(np.int32)

        
    save_image_path = f"results/{name}"
    save_result(ori_x, ori_y, y_pred, save_image_path)

100%|██████████| 500/500 [02:05<00:00,  3.99it/s]


In [20]:
import shutil
shutil.make_archive('./results', 'zip', 'results')

'/kaggle/working/results.zip'

In [21]:
from IPython.display import FileLink
FileLink(r'./results.zip')

/kaggle/working/results.zip